In [30]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

# Load the parquet file into a pandas dataframe
train_data = pd.read_parquet("C:/Users/Divine/Downloads/train-00000-of-00002-9767f7576a73674b.parquet")
test_data = pd.read_parquet("C:/Users/Divine/Downloads/test-00000-of-00004-794076eacfe4b3b3.parquet")

In [61]:
X_train = np.array(train_data['image'])
y_train = np.array(train_data['label'])
X_test = np.array(test_data['image'])
y_test = np.array(test_data['label'])

In [70]:
train_data

,id,image,label
0,0.jpg,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,1
1,1.jpg,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,1
2,2.jpg,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,1
3,3.jpg,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0
4,4.jpg,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,1
...,...,...,...
9304,9304.jpg,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,1
9305,9305.jpg,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0
9306,9306.jpg,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,1
9307,9307.jpg,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,0


In [65]:
def preprocess_image(image_dict):
    image_bytes = image_dict["bytes"]
    image_array = np.frombuffer(image_bytes, np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    # image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize image to a fixed size (e.g. 28x28)
    resized_image = cv2.resize(gray_image, (28, 28))
    # Normalize the pixel values to be between 0 and 1
    normalized_image = resized_image / 255.0
    # Add a third dimension to represent the color channels (1 for grayscale)
    final_image = np.expand_dims(normalized_image, axis=-1)
    return final_image
# Preprocess the training and testing images
X_train_processed = np.array([preprocess_image(images) for images in X_train])
X_test_processed = np.array([preprocess_image(images) for images in X_test])


In [71]:
# CNN model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Model Compilation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
model.fit(X_train_processed, y_train, epochs=20, batch_size=52, validation_split=0.2)

# Evaluation
loss, accuracy = model.evaluate(X_test_processed, y_test)

Epoch 1/20
144/144 [==============================] - 4s 21ms/step - loss: 0.5360 - accuracy: 0.7138 - val_loss: 0.4440 - val_accuracy: 0.7642
Epoch 2/20
144/144 [==============================] - 3s 20ms/step - loss: 0.4077 - accuracy: 0.8091 - val_loss: 0.3797 - val_accuracy: 0.8195
Epoch 3/20
144/144 [==============================] - 3s 20ms/step - loss: 0.3590 - accuracy: 0.8336 - val_loss: 0.3558 - val_accuracy: 0.8255
Epoch 4/20
144/144 [==============================] - 3s 20ms/step - loss: 0.3294 - accuracy: 0.8489 - val_loss: 0.3313 - val_accuracy: 0.8383
Epoch 5/20
144/144 [==============================] - 3s 20ms/step - loss: 0.3055 - accuracy: 0.8610 - val_loss: 0.3200 - val_accuracy: 0.8421
Epoch 6/20
144/144 [==============================] - 3s 20ms/step - loss: 0.2904 - accuracy: 0.8663 - val_loss: 0.3142 - val_accuracy: 0.8518
Epoch 7/20
144/144 [==============================] - 3s 20ms/step - loss: 0.2723 - accuracy: 0.8787 - val_loss: 0.3181 - val_accuracy: 0.8362